In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
from pandas_datareader.data import DataReader
from datetime import date

In [2]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py
import plotly.graph_objs as go
from plotly.grid_objs import Grid, Column
from plotly.tools import FigureFactory as FF
import time

In [3]:
cd ..

/home/wbaik/Code/data_science/temp_folder


In [4]:
data = pd.read_csv('10K_data.csv')

In [5]:
data.keys()

Index(['Unnamed: 0', 'Symbol', 'Wikipedia Link', 'Name', 'Sector',
       'SEC Filings Link', 'CIK', 'Latest 10K', 'Enterprise Value',
       'Trailing P/E', 'Enterprise Value/Revenue', 'Enterprise Value/EBITDA',
       'Revenue', 'EBITDA', 'Diluted EPS', 'Market Cap (intraday)',
       'Forward P/E', 'Beta', '50-Day Moving Average',
       '200-Day Moving Average', 'Avg Vol (3 month)', 'Avg Vol (10 day)',
       'Shares Outstanding'],
      dtype='object')

In [5]:
# iterables are what to retrieve from the given_dataframe 
# to cut into different `size`
def add_columns_by_cuts(iterables, given_dataframe, size):
    
    def create_labels(size, name):
        return ['{} >= {}%'.format(name, i/size*100) for i in range(size)]
    
    new_dataframe = None
    
    for cur in iterables:
    
        labels = create_labels(size, cur)
        
        cut = pd.qcut(given_dataframe[cur], size, precision=2, labels=labels)
        cut.name = cur + ' % Rank'
            
        if new_dataframe is None:
            new_dataframe = pd.concat([given_dataframe, cut], axis=1)
        else:
            new_dataframe[cut.name] = cut
    
    return new_dataframe



In [6]:
def draw_from_cuts(iterables_used_for_cuts, result_from_cuts, size_of_cuts):
    
    names_used_for_cuts = []
    for i in iterables_used_for_cuts:
        names_used_for_cuts.append(i + ' % Rank')
    
    # mean() should be replaced by some function
    table = result_from_cuts.groupby(names_used_for_cuts[:-1]).mean()\
            [iterables_used_for_cuts[-1]].unstack().fillna(0).\
            unstack().reset_index()
    table.columns = ['x', 'y', 'z']

    colors = list(table['z']/max(table['z']) * 100)

    my_trace = [
        go.Scatter(
            x = table['x'],
            y = table['y'],
            mode = 'markers',
            marker = dict(
                size = table['z'],
                sizemode='area',
                sizeref=2.*max(table['z'])/(100.**2),
                sizemin=4,
                color=colors,
                showscale=True
            )
        )
    ]
    
    init_notebook_mode(connected=True)
    py.iplot(my_trace)

In [7]:
ret = add_columns_by_cuts(['Revenue', 'EBITDA', 'Enterprise Value'], data, 5)

draw_from_cuts(['Revenue', 'EBITDA', 'Enterprise Value'], ret, 5)

In [8]:
grouped_by_sectors = data.groupby('Sector')
sectors = dict(list(grouped_by_sectors))

In [9]:
sectors.keys()

dict_keys(['Consumer Discretionary', 'Consumer Staples', 'Energy', 'Financials', 'Health Care', 'Industrials', 'Information Technology', 'Materials', 'Real Estate', 'Telecommunication Services', 'Utilities'])

In [10]:
it = sectors['Information Technology']

ret = add_columns_by_cuts(['Revenue', 'EBITDA', 'Enterprise Value'], it, 5)

draw_from_cuts(['Revenue', 'EBITDA', 'Enterprise Value'], ret, 5)

In [11]:
energy = sectors['Energy']

ret = add_columns_by_cuts(['Revenue', 'EBITDA', 'Enterprise Value'], energy, 5)

draw_from_cuts(['Revenue', 'EBITDA', 'Enterprise Value'], ret, 5)

In [12]:
from plotly.graph_objs import Scatter3d

df = data

trace1 = Scatter3d(
    x=df['Sector'],
    y=df['Enterprise Value'],
    z=df['Beta'],
    mode='markers',
    marker=dict(
        sizemode='diameter',
        sizeref=min(df['Forward P/E'])/3,
        size=df['Forward P/E'],
        color = df['Forward P/E'],
        colorscale = 'Viridis',
        
    )
)

to_plot=[trace1]
layout=dict(height=800, width=800, title='Examining Sectors, Enterprise Value, and Beta for its Forward P/E')
fig=dict(data=to_plot, layout=layout)
py.iplot(fig, filename='3DBubble')

In [13]:
ret.keys()

Index(['Unnamed: 0', 'Symbol', 'Wikipedia Link', 'Name', 'Sector',
       'SEC Filings Link', 'CIK', 'Latest 10K', 'Enterprise Value',
       'Trailing P/E', 'Enterprise Value/Revenue', 'Enterprise Value/EBITDA',
       'Revenue', 'EBITDA', 'Diluted EPS', 'Market Cap (intraday)',
       'Forward P/E', 'Beta', '50-Day Moving Average',
       '200-Day Moving Average', 'Avg Vol (3 month)', 'Avg Vol (10 day)',
       'Shares Outstanding', 'Revenue % Rank', 'EBITDA % Rank',
       'Enterprise Value % Rank'],
      dtype='object')

In [14]:
trace1 = go.Scatter(
    x = data['Revenue'],
    y = data['EBITDA'],
    name = 'Below',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = data['Beta'],
        line = dict(
            width = 2,
        )
    )
)

py.iplot([trace1])

In [15]:
trace1 = go.Box(
    x = data['50-Day Moving Average']/data['200-Day Moving Average'],
    boxpoints='all',
    jitter=0.3,
    pointpos=-1.8
)


py.iplot([trace1])

### Retrieve Fundamental Data

In [16]:
def save_fundamentals_to_csv():
    import os
    
    def get_all_the_data():

        import good_morning as gm
        kr = gm.KeyRatiosDownloader()

        key_ratios = {}
        for i in data['Symbol']:
            if i not in files:
                try:
                    key_ratios[i] = kr.download(i)
                    time.sleep(1)
                except ValueError:
                    print('Value Error raised for {}'.format(i))
                except:
                    print('Unknown Error')
        return key_ratios
    
    key_ratios = get_all_the_data()
    files = os.listdir('./fundamental_data/')

    for ticker in key_ratios:
        if ticker not in files:
            pd.concat(key_ratios[ticker]).T\
                .to_csv('./fundamental_data/{}'.format(ticker))
    

In [17]:
def retrieve_fudamentals_from_csv():
    import os
    list_of_files = os.listdir('./fundamental_data/')
    
    # We've got to make this as an object... this is a terrible way
    data = pd.read_csv('10K_data.csv')
    symbols = [ticker for ticker in data['Symbol']]
    ret = {}
    for filename in list_of_files:
        if filename not in symbols:
            continue
        ret[filename] = pd.read_csv('./fundamental_data/{}'.format(filename))
    return ret

In [18]:
def get_revenue_related_data():
    import re
    
    ret = {}
    fundamental_dict = retrieve_fudamentals_from_csv()

    for i, ticker in enumerate(fundamental_dict):
        # Just using this for a test
        if i == 3: break
        
        current_df = fundamental_dict[ticker]
        revenue_columns = ['Period']
        # this is completely unncessary...
        # revenue_columns += [i for i in current_df.columns
        #                    if re.search(r'Revenue', i) is not None]
        revenue_columns += [i for i in current_df.columns
                            if 'Revenue' in i and 'Revenue' != i]
        
        ret[ticker] = pd.concat([current_df[[col]] for col in revenue_columns], axis=1)
    
    return ret  

In [19]:
symbols = [sym for sym in data['Symbol']]

def save_all_stock_data(symbols):
    from pandas_datareader import data
    from datetime import datetime
    from pathlib import Path
    
    for ticker in symbols:
        # Define which online source one should use
        data_source = 'morningstar'

        start_date = '2010-01-01'
        end_date = '2018-04-01'

        # User pandas_reader.data.DataReader to load the desired data. As simple as that.
        file_path = \
            Path('./fundamental_data/{}_price'.format(ticker))
            
        if not file_path.is_file():
            try:
                panel_data = data.DataReader(ticker, data_source, start_date, end_date)
                panel_data.to_csv(file_path)
                time.sleep(5)
            except:
                print('Unkown Exception from {}'.format(ticker))

# save_all_stock_data(symbols)

In [20]:
fundamental_dict = retrieve_fudamentals_from_csv()

dict_keys(['ANDV', 'NOC', 'MSCI', 'JWN', 'EL', 'BLL', 'AOS', 'ED', 'RRC', 'BA', 'PFE', 'DHR', 'CLX', 'STX', 'JCI', 'LRCX', 'UTX', 'DIS', 'DE', 'MAR', 'UAL', 'BWA', 'MCD', 'KORS', 'HPE', 'CRM', 'ISRG', 'TWX', 'XLNX', 'ETN', 'PEP', 'EOG', 'CNC', 'DHI', 'NWL', 'HOLX', 'CI', 'PH', 'AZO', 'MDT', 'KMB', 'RMD', 'AKAM', 'ORLY', 'EA', 'IVZ', 'CBS', 'HUM', 'AMT', 'RHT', 'NEE', 'PXD', 'FCX', 'PAYX', 'MDLZ', 'FBHS', 'XEL', 'LNT', 'TSN', 'JNPR', 'ULTA', 'WY', 'PSX', 'HD', 'NDAQ', 'IBM', 'O', 'REGN', 'VRSK', 'GOOGL', 'DOV', 'PNW', 'CA', 'PCAR', 'MHK', 'KIM', 'ADS', 'BEN', 'MON', 'AAL', 'KLAC', 'ALL', 'ANSS', 'MGM', 'BXP', 'XYL', 'SYK', 'GILD', 'REG', 'CHTR', 'LEG', 'ABC', 'DG', 'INFO', 'KMX', 'CVS', 'EXR', 'IRM', 'VTR', 'AME', 'SEE', 'QRVO', 'WELL', 'DPS', 'WRK', 'JNJ', 'FAST', 'DRE', 'FL', 'LYB', 'HON', 'EXC', 'WYN', 'GPS', 'RHI', 'PM', 'EMR', 'T', 'ALB', 'CBRE', 'AVGO', 'SRCL', 'ROST', 'HCA', 'MU', 'GPN', 'UDR', 'CAH', 'NLSN', 'RTN', 'PKI', 'NKE', 'NUE', 'INTC', 'KR', 'AFL', 'AAPL', 'SRE', 'FISV',

In [22]:
def remove_unreliable(fundamental_dict):
    
    def unreliable_tickers():
        unreliable_data = set()

        for ticker in fundamental_dict:
            df = fundamental_dict[ticker]
            revenue_column = df.columns[1]
            mean = df[revenue_column].mean()
            if mean < 100 or df[revenue_column].isna().any():
                unreliable_data.add(ticker)

        return unreliable_data  
    
    unreliables = unreliable_tickers()
    
    for ticker in unreliables:
        fundamental_dict.pop(ticker, None)
    
    return fundamental_dict

In [31]:
print('Originally fundamental keys length: {}'.format(len(fundamental_dict.keys())))
fundamental_dict = remove_unreliable(fundamental_dict)
print('After the removal, length:{}'.format(len(fundamental_dict.keys())))

symbols = [sym for sym in fundamental_dict.keys()]

Originally fundamental keys length: 274
After the removal, length:274


In [32]:
def given_fundamental_dict_create_columns(given_dict_with_year,
                                          to_compute=[np.mean, np.var]):
    from toolz import curry
    from postgres_utils import PostgresPandas
    
    @curry
    def run_query_given_ticker_and_time(ticker, func, year):
        
        start = '{}-12-31'.format(year-1)
        end = '{}-01-01'.format(year+1)

        query = '''
        select close from {} where \'{}\' < date and date < \'{}\'
        '''.format(ticker, start, end)

        try:
            ret = ppd.run_query(query)
            return ret.agg(func)
        except:
            print('Query failed for {}: {}'.format(ticker, year))
            return np.NaN
    
    ppd = PostgresPandas()

    for ticker in given_dict_with_year:
        # This modifies df inplace, generally, but not sure 
        #     if that's always the case... Assume it's a pointer
        df = given_dict_with_year[ticker]
        
        for func in to_compute:
            
            curried = run_query_given_ticker_and_time(ticker, func)
            
            # prefer renaming the column to something else...
            name_of_the_function = str(func).split()[1]
            df[name_of_the_function] = df['Period'].apply(curried)
    
    del ppd
    
    return given_dict_with_year
    

In [33]:
temp_dict = {ticker: fundamental_dict[ticker] 
             for ticker in np.random.choice(symbols, 10) if ticker in fundamental_dict}

In [34]:
temp_dict.keys()

dict_keys(['MMC', 'EMR', 'JWN', 'XLNX', 'CELG', 'KLAC', 'RSG', 'CL', 'IDXX', 'UHS'])

In [35]:
column_created = given_fundamental_dict_create_columns(temp_dict)

In [36]:
column_created.keys()

dict_keys(['MMC', 'EMR', 'JWN', 'XLNX', 'CELG', 'KLAC', 'RSG', 'CL', 'IDXX', 'UHS'])

In [58]:
def revenue_against_stock_price():
    plot_data = []
    
    for ticker in column_created:
        plot_data.append(
            go.Scatter(
                x = column_created[ticker].iloc[:, 0],
                y = column_created[ticker].iloc[:, 1].dropna(),
                mode = 'markers',
                marker = dict(
                    size = column_created[ticker].iloc[:, -2].dropna() / 5,
                    color = column_created[ticker].iloc[:, -1].dropna(),
                    line = dict(width = 2)
                )
            )
        )
        
    py.iplot(plot_data)

In [59]:
revenue_against_stock_price()

In [66]:
trace1 = Scatter3d(
    x=df['Sector'],
    y=df['Enterprise Value'],
    z=df['Beta'],
    mode='markers',
    marker=dict(
        sizemode='diameter',
        sizeref=min(df['Forward P/E'])/3,
        size=df['Forward P/E'],
        color = df['Forward P/E'],
        colorscale = 'Viridis',
        
    )
)

def revenue_price_3d():
    plot_data = []
    
    for ticker in column_created:
        plot_data.append(
            go.Scatter3d(
                x = column_created[ticker].iloc[:, 0],
                y = column_created[ticker].iloc[:, 1].dropna(),
                z = column_created[ticker].iloc[:, -2].dropna(),
                mode = 'markers',
                marker = dict(
                    sizemode='diameter',
                    sizeref=min(column_created[ticker].iloc[:, -2].dropna())/5,
                    size = column_created[ticker].iloc[:, -2].dropna(),
                    color = column_created[ticker].iloc[:, 0].dropna(),
                    colorscale='Viridis'
                )
            )
        )
        
    py.iplot(plot_data)

In [67]:
revenue_price_3d()

In [ ]:
k